In [ ]:
import os
from pathlib import Path

from atomate2.jdftx.io.JDFTXInfile import JDFTXInfile
from atomate2.jdftx.io.JDFTXOutfile import JDFTXOutfile

In [4]:
from typing import Any, List
def flatten_list(tag: str, list_of_lists: List[Any]) -> List[Any]:
    """Flattens list of lists into a single list, then stops"""
    if not isinstance(list_of_lists, list):
        raise ValueError(f"{tag}: You must provide a list to flatten_list()!")
    while any([isinstance(x, list) for x in list_of_lists]):
        list_of_lists = sum(list_of_lists, [])
    return list_of_lists

def flatten_list(tag: str, list_of_lists: List[Any]) -> List[Any]:
    """Flattens list of lists into a single list, then stops"""
    if not isinstance(list_of_lists, list):
        raise ValueError(f"{tag}: You must provide a list to flatten_list()!")
    flist = []
    for v in list_of_lists:
        if isinstance(v, list):
            flist.extend(flatten_list(tag, v))
        else:
            flist.append(v)
    return flist


In [5]:

flatten_list("test", [True, True, [True, True], True])


[True, True, True, True, True]

In [2]:
jif = JDFTXInfile.from_file(Path(os.getcwd()) / "example_files" / "example_sp.in")

here
here


In [ ]:
jif.get_text_list()

In [ ]:
test = JDFTXOutfile.from_file(Path(os.getcwd()) / "example_files" / "example_sp.out")
print(test[-1].jstrucs.iter_type)
print(test[-1].t_s)

In [ ]:
from atomate2.jdftx.io.JDFTXInfile_master_format import *

In [ ]:
tag_ex = "fluid-anion"

instance = JDFTXInfile()
lines = [
    "fluid-anion Cl- 0.5",
    "lattice \\",
    "1.0 0.0 0.0 \\",
    "1.0 0.0 0.0 \\",
    "1.0 0.0 0.0",
    "latt-move-scale 0 0 0",
    "ion Cl 0.0 0.0 0.0 1",
    "ion-species GBRV_v1.5/$ID_pbe_v1.uspp",
]
# lines = instance._gather_tags(lines)

test = JDFTXInfile.from_str("\n".join(lines))